In [45]:
import os
import re
import random
import pandas as pd
import numpy as np

In [35]:
data = pd.read_csv('../data/raw/Training/Phase2Answers/Phase2Answers-1a.txt', sep='\t')
data = data.rename({'# pair1': 'pair1'}, axis='columns')

options_df = data[['pair1', 'pair2', 'pair3', 'pair4']]
options_df = options_df.drop_duplicates().reset_index(drop=True)

In [52]:
answers = pd.read_csv('../data/external/gpt_answers_train_1a.txt', sep=', ', names=['most_illustrative', 'least_illustrative'])
answers['most_illustrative'] = answers['most_illustrative'].apply(lambda x: x.replace('Most illustrative: ', '').replace('"', ''))
answers['least_illustrative'] = answers['least_illustrative'].apply(lambda x: x.replace('Least illustrative: ', '').replace('"', ''))

answers = pd.concat((answers, options_df), axis=1)

/tmp/ipykernel_6199/3095651868.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  answers = pd.read_csv('../data/external/gpt_answers_train_1a.txt', sep=', ', names=['most_illustrative', 'least_illustrative'])


In [53]:
answers

,most_illustrative,least_illustrative,pair1,pair2,pair3,pair4
0,vegetable:carrot,currency:dollar,oak:tree,vegetable:carrot,tree:oak,currency:dollar
1,dog:poodle,sweater:knit,appliance:refrigerator,sweater:knit,dog:poodle,tree:oak
2,mammal:porpoise,bread:wheat,furniture:chair,furniture:couch,mammal:porpoise,bread:wheat
3,currency:dollar,dollar:currency,currency:dollar,animal:pig,dollar:currency,bird:swan
4,politician:senator,wheat:bread,automobile:van,wheat:bread,weather:rain,politician:senator
...,...,...,...,...,...,...
98,bush:astilbe,hair:brown,hair:brown,tree:ash,bush:azalea,bush:astilbe
99,automobile:van,pen:ballpoint,automobile:van,bird:swan,pen:ballpoint,animal:pig
100,dog:poodle,sweater:knit,wheat:bread,sweater:knit,dog:pet,dog:poodle
101,dog:chihuahua,politician:senator,dog:chihuahua,bush:azalea,politician:senator,animal:carabao


In [28]:
data

,pair1,pair2,pair3,pair4,least_illustrative,most_illustrative,user_selected_relation
0,oak:tree,vegetable:carrot,tree:oak,currency:dollar,tree:oak,oak:tree,Y is a kind/type/instance of X
1,oak:tree,vegetable:carrot,tree:oak,currency:dollar,oak:tree,tree:oak,Y is a kind/type/instance of X
2,oak:tree,vegetable:carrot,tree:oak,currency:dollar,oak:tree,vegetable:carrot,Y is a kind/type/instance of X
3,oak:tree,vegetable:carrot,tree:oak,currency:dollar,oak:tree,vegetable:carrot,Y is a kind/type/instance of X
4,oak:tree,vegetable:carrot,tree:oak,currency:dollar,oak:tree,tree:oak,Y is a kind/type/instance of X
...,...,...,...,...,...,...,...
501,color:red,vegetable:carrot,hair:brown,weather:rain,weather:rain,color:red,Y is a kind/type/instance of X
502,color:red,vegetable:carrot,hair:brown,weather:rain,weather:rain,color:red,Y is a kind/type/instance of X
503,color:red,vegetable:carrot,hair:brown,weather:rain,hair:brown,color:red,Y is a kind/type/instance of X
504,color:red,vegetable:carrot,hair:brown,weather:rain,weather:rain,vegetable:carrot,Y is a kind/type/instance of X


In [27]:
least_data

pair1           pair2                   pair3               pair4                             
animal:carabao  appliance:refrigerator  mammal:porpoise     weather:rain    weather:rain          2
                                                                            animal:carabao        2
                                                                            mammal:porpoise       1
                footwear:boot           politician:senator  automobile:van  politician:senator    3
                                                                            automobile:van        2
                                                                                                 ..
weather:rain    animal:carabao          footwear:boot       clothing:shirt  weather:rain          1
wheat:bread     currency:dollar         fruit:grape         bush:azalea     wheat:bread           5
                sweater:knit            dog:pet             dog:poodle      dog:pet               3
     

In [16]:
def pairqnum2least(question_data):
    return question_data['least_illustrative'].value_counts()

def pairqnum2most(question_data):
    return question_data['most_illustrative'].value_counts()

In [17]:
least_data = data.groupby(['pair1', 'pair2', 'pair3', 'pair4']).apply(lambda x: pairqnum2least(x))
most_data = data.groupby(['pair1', 'pair2', 'pair3', 'pair4']).apply(lambda x: pairqnum2most(x))

In [79]:
def maxdiff(answer):
    least_ans = answer['least_illustrative']
    most_ans = answer['most_illustrative']
    pairs = answer[['pair1', 'pair2', 'pair3', 'pair4']].values.ravel()
    least_pairs = tuple(np.append(pairs, least_ans))
    most_pairs = tuple(np.append(pairs, most_ans))

    if least_pairs in least_data.index:
        votes_guess_least = least_data.loc[least_pairs]
    else:
        votes_guess_least = 0
    if most_pairs in most_data.index:
        votes_guess_most = most_data.loc[most_pairs]
    else:
        votes_guess_most = 0
    max_votes_least = 0
    max_votes_most  = 0

    for pair in pairs:
        curr_pairs = tuple(np.append(pairs, pair))
        if curr_pairs in least_data.index:
            num_votes_least = least_data.loc[curr_pairs]
        else:
            num_votes_least = 0
        if curr_pairs in most_data.index:
            num_votes_most = most_data.loc[curr_pairs]
        else:
            num_votes_most = 0
        if num_votes_least > max_votes_least:
            max_votes_least = num_votes_least
        if num_votes_most > max_votes_most:
            max_votes_most = num_votes_most

    num_least_right = num_least_wrong = num_most_right = num_most_wrong = 0

    if votes_guess_least == max_votes_least:
        num_least_right += 1
    else:
        num_least_wrong += 1
    if votes_guess_most == max_votes_most:
        num_most_right += 1
    else:
        num_most_wrong += 1

    ret_df = answer.copy()
    ret_df['num_least_right'] = num_least_right
    ret_df['num_least_wrong'] = num_least_wrong
    ret_df['num_most_right'] = num_most_right
    ret_df['num_most_wrong'] = num_most_wrong

    return ret_df

In [82]:
answers_with_scores = answers.apply(lambda x: maxdiff(x), axis=1)

In [87]:
acc_most = 100 * answers_with_scores.num_most_right.sum() / len(answers)
acc_least = 100 * answers_with_scores.num_least_right.sum() / len(answers)
acc_all = 100 * (answers_with_scores.num_most_right.sum() + answers_with_scores.num_least_right.sum()) / (2 * len(answers))

In [88]:
acc_all

38.83495145631068